# Lecture 06 — Market Research and Analysis

**テーマ:** Ward 法でクラスタリングを行い、dendrogram を見て、cut して cluster を作る。  
**目的:** cluster を元データに merge し、頻度分布と基礎記述を確認する。


## 0) データと準備

- `c.unico_1` は 8 つの opinion 変数 `d10_1`〜`d10_8` を持つ。  
- `id` は個体の ID。  
- `c.unico_2` は `unico_1` に `cluster` を merge した完成版。


In [ ]:
* path for students;
libname c 'C:\Camillo_clabe';
* path for teacher (Furio);
libname c '\\MaterialeDocente\Camillo_clabe';


## 1) Ward 法の考え方（要点）

- Within variability を最小化、Between variability を最大化する。  
- 新しい点をどの group に入れるかは、内部分散の増分が最も小さい方を選ぶ。  
- これにより、各 cluster は中で似ていて、外とは離れる。


## 2) Dendrogram の作成

- まず参考に method=single を動かし、次に本命の method=ward を行う。  
- `var d10_:;` は `d10_` で始まる全ての列をまとめて指定する SAS shorthand。


In [ ]:
/* 参考: Single linkage での tree（比較用） */
proc cluster data=c.unico_1 method=single outtree=c.tree;
  var d10_:;
  id id;
run;

proc tree data=c.tree;
  id id;
run;

/* 本命: Ward 法の tree を作る */
proc cluster data=c.unico_1 method=ward outtree=c.tree_1;
  var d10_:;
  id id;
run;

/* Dendrogram を表示 */
proc tree data=c.tree_1;
  id id;
run;

## 3) クラスタ数の決定と cut

- dendrogram の fusion distance の大きなジャンプで切る。  
- 授業では 3 clusters を採用。`PROC TREE` で `out=` と `nclusters=` を使う。


In [ ]:
/* cut して cluster 割当てを作る */
proc tree data=c.tree_1 noprint out=c.cluster nclusters=3;
  id id;
run;

## 4) cluster を元データへ merge

- merge にはキーが必要。ここでは id。  
- PROC SORT で事前に by id の sort を行うこと。  
- ファイルを開いたままにすると sort が失敗するので注意。


In [ ]:
/* 両データを id で sort */
proc sort data=c.cluster;  by id; run;
proc sort data=c.unico_1;  by id; run;

/* merge して完成版 c.unico_2 を作る */
data c.unico_2;
  merge c.unico_1 c.cluster;
  by id;
run;

## 5) cluster のサイズと構成比を確認

- PROC FREQ の `tables cluster;` で件数と比率を見る。  
- 欠損がある行は自動で除外されることがある。


In [ ]:
/* cluster の分布（件数と%） */
proc freq data=c.unico_2;
  tables cluster;
run;

## 6) 欠損値の注意

- opinion の一部が欠損だと、クラスタリングから外れることがある。  
- 授業では 784 から 782 に減る例があった。  
- 後で missing value treatment を検討する。


## 7) 外部カテゴリとの関係を見る準備（導入）

- cluster は categorical、sex も categorical。  
- 両者の関係は two-way table で確認する。  
- independence の仮説の下、expected は `E_ij = (row total_i × column total_j) / N`。


In [ ]:
/* two-way table（導入、期待度数と Chi-square を出す） */
proc freq data=c.unico_2;
  tables cluster*sex / expected chisq;
run;

## 8) opinion 変数の基礎記述

- d10_1〜d10_8 の全体平均を見て基準を持つ。  
- 次に cluster 別平均を見て、over-characterized か under-characterized かを掴む。


In [ ]:
/* 全体の基礎記述 */
proc means data=c.unico_2 mean stddev min max;
  var d10_1-d10_8;
  title "Overall means for opinion variables";
run;

/* cluster 別の基礎記述 */
proc means data=c.unico_2 mean stddev min max;
  class cluster;
  var d10_1-d10_8;
  title "Cluster means for opinion variables";
run;

## まとめ

- Ward 法は within variability を抑え、between variability を広げる。  
- Dendrogram を見て、適切な cut を選ぶ。授業では 3 clusters。  
- PROC TREE で割当てを作り、id で merge。  
- PROC FREQ でサイズを確認、PROC MEANS で基礎記述。  
- two-way table と expected は Lecture 07 に続く分析の土台になる。
